In [2]:
import zipfile as zf
import glob
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
from statistics import median

import warnings
warnings.filterwarnings("ignore")

# unzip file: only need to do it once

In [3]:
def unzip(path):
    files = zf.ZipFile(path,'r')
    files.extractall()
    files.close()

# load data

In [4]:
def input_data(name, path, Id_range = None, add_Date = False):
    files = glob.glob(path)
    if add_Date:
        dfs = []
        for f in files:
            df = pd.read_csv(f, sep=",")
            df['Date'] = f[-12:-4]
            df = df[df.Id.isin(Id_range)]
            dfs.append(df)
    else:
        dfs = [pd.read_csv(f, sep=",") for f in files]    
    data = pd.concat(dfs,ignore_index=True)
    data.Date = data.Date.apply(str)
    return data

In [5]:
def load_data(path):
    ret = input_data("return", path+"return\\*.csv")

    Id_range = ret.Id.unique()
    risk = input_data("risk", path+"risk\\*.csv", Id_range = Id_range, add_Date = True)
    
    ret = pd.merge(ret,risk,how="left",on=['Date', 'Id'])
    
    mdv = input_data("mdv",  path+"mdv\\*.csv")
    volume = input_data("price_volume",  path+"price_volume\\*.csv")    
    shout = input_data("shout",  path+"shout\\*.csv")

    ret = pd.merge(ret,shout,how="left",on=['Date', 'Id'])    
    ret = pd.merge(ret,mdv,how="left",on=['Date','Id'])    
    ret = pd.merge(ret,volume,how="left",on=['Date','Time', 'Id'])
    
    print("ret shape: ", ret.shape)
    
    return ret

# missing value

In [6]:
def compare_ffill(row):
    if row.Time[:2]!='10':
        if row.ResidualCumReturn_ffill==float("nan") or row.ResidualCumReturn_ffill<row.ResidualNoWinsorCumReturn:
            row.ResidualCumReturn_ffill = row.ResidualNoWinsorCumReturn
        if row.RawCumReturn_ffill==float("nan") or row.RawCumReturn_ffill<row.RawNoWinsorCumReturn:
            row.RawCumReturn_ffill = row.RawNoWinsorCumReturn
        if row.CumVol_ffill==float("nan") or row.CumVol_ffill<row.CumVolume:
            row.CumVol_ffill = row.CumVolume
    return row

In [7]:
def missing_value(ret, na_type = 1):
    # get the number of missing data points per column
    missing_values_count = ret.isnull().sum()
    print("missing_values_count: \n", missing_values_count)
    
    # percent of data that is missing
    total_cells = np.product(ret.shape)
    total_missing = missing_values_count.sum()

    # percent of data that is missing
    percent_missing = (total_missing/total_cells) * 100
    print("percent_missing: ", percent_missing)

    
    if na_type == 2:
        # fillna
        ret = ret.groupby(['Id']).apply(lambda x: x.ffill())  
    elif na_type == 3:
        # fillna_cumulative
        ret2 = pd.DataFrame(index=ret.index)
        ret2[['ResidualCumReturn_ffill','RawCumReturn_ffill','CumVol_ffill']] = ret.groupby(['Time', 'Id'])['ResidualNoWinsorCumReturn', 
                                                                                                            'RawNoWinsorCumReturn', 
                                                                                                            'CumVolume'].apply(lambda x: x.ffill())
        ret = ret.groupby(['Id']).apply(lambda x: x.ffill())  
        ret[['ResidualCumReturn_ffill','RawCumReturn_ffill','CumVol_ffill']] = ret2[['ResidualCumReturn_ffill',
                                                                                      'RawCumReturn_ffill',
                                                                                      'CumVol_ffill']]
        ret = ret.apply(lambda row: compare_ffill(row), axis = 'columns')  
        ret = ret.drop(columns=['ResidualCumReturn_ffill','RawCumReturn_ffill','CumVol_ffill'])
        new_names = ['ResidualNoWinsorCumReturn','RawNoWinsorCumReturn','CumVolume']
        old_names = ['ResidualCumReturn_ffill','RawCumReturn_ffill','CumVol_ffill']
        ret.rename(columns=dict(zip(old_names, new_names)), inplace=True)

    # dropna    
    ret = ret.dropna()
    
    return ret

# winsorization

In [8]:
def winsorization(ret):
    # extract numeric columns
    numeric_columns = list(ret.dtypes[ret.dtypes != "object"].index)
    numeric_columns.remove('IsOpen')
    
    for feature in numeric_columns:   
        ret[str(feature+' winsorized')] = np.zeros(ret.shape[0])
        date_range = ret.Date.unique()

        for date in date_range:  
            # compute limits
            mad = ret[ret.Date==date][feature].mad()
            med = median(ret[ret.Date==date][feature])        
            llimit, ulimit = med - 5*mad, med + 5*mad

            # Create winsorized versions of the features with new column names        
            ret[str(feature+' winsorized')][ret.Date==date] = np.clip(ret[ret.Date==date][feature],llimit, ulimit)
        
    ret.to_csv('merged_data.csv',index=False)  
    return ret

In [15]:
def data_preprocess(path, unzip_type = False):
    if unzip_type:
        unzip(path+"train.zip")
    ret = load_data(path+"train\\")
    ret = missing_value(ret, na_type = 3)
    ret = winsorization(ret)
    return ret

# Main

In [16]:
data_preprocess(r"C:\Users\Lenovo\Python_workshop\project\\", unzip_type = True)

ret shape:  (2448284, 11)
missing_values_count: 
 Date                              0
Time                              0
Id                                0
ResidualNoWinsorCumReturn         0
RawNoWinsorCumReturn              0
estVol                            0
SharesOutstanding                84
MDV_63                            6
CleanMid                     254473
CumVolume                    254473
IsOpen                       254473
dtype: int64
percent_missing:  2.8350432013160685


,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn,estVol,SharesOutstanding,MDV_63,CleanMid,CumVolume,IsOpen,ResidualNoWinsorCumReturn winsorized,RawNoWinsorCumReturn winsorized,estVol winsorized,SharesOutstanding winsorized,MDV_63 winsorized,CleanMid winsorized,CumVolume winsorized
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0,1.0,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0,1.0,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0,1.0,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0,1.0,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0,1.0,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448279,20171229,17:30:00.000,ID00GVXK059,-0.002841,-0.002123,0.249899,727.820,6115657.0,6.719321,501410.0,1.0,-0.002841,-0.002123,0.249899,727.820,6115657.0,6.719321,501410.0
2448280,20171229,17:30:00.000,ID00HFWT7M4,0.000843,-0.005238,0.104843,170.875,55438068.0,234.749070,18108.0,1.0,0.000843,-0.005238,0.104843,170.875,55438068.0,216.695569,18108.0
2448281,20171229,17:30:00.000,ID00HRW4917,0.033315,0.031948,0.192046,27.126,2810998.5,58.018875,13716.0,1.0,0.024701,0.031163,0.192046,27.126,2810998.5,58.018875,13716.0
2448282,20171229,17:30:00.000,ID00HTQM455,0.011946,0.009285,0.130896,100.000,3763925.0,53.096570,3374.0,1.0,0.011946,0.009285,0.130896,100.000,3763925.0,53.096570,3374.0


In [18]:
df = pd.read_csv(r"C:\Users\Lenovo\Python workshop\project\merged_data.csv", sep=",")
df.head()

,Date,Time,Id,ResidualNoWinsorCumReturn,RawNoWinsorCumReturn,estVol,SharesOutstanding,MDV_63,CleanMid,CumVolume,IsOpen,ResidualNoWinsorCumReturn winsorized,RawNoWinsorCumReturn winsorized,estVol winsorized,SharesOutstanding winsorized,MDV_63 winsorized,CleanMid winsorized,CumVolume winsorized
0,20140102,10:00:00.000,IDZQ0003GF7,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0,1.0,0.008795,0.008043,0.093123,110.580,4321539.0,32.009570,30851.0
1,20140102,10:00:00.000,IDZQ00047M8,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0,1.0,-0.007513,-0.008949,0.129865,449.750,7728599.0,5.929723,100377.0
2,20140102,10:00:00.000,IDZQ00048R1,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0,1.0,0.002052,0.001240,0.151308,227.481,34529812.0,46.881290,118814.0
3,20140102,10:00:00.000,IDZQ0005R34,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0,1.0,-0.007269,-0.006079,0.122016,575.375,2318102.2,6.155510,57972.0
4,20140102,10:00:00.000,IDZQ0005SC2,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0,1.0,-0.004682,-0.009140,0.066838,839.394,66501836.0,27.234093,399523.0
